In [ ]:
"""
Following this tutorial by Andrej Karpathy: https://youtu.be/kCc8FmEb1nY
"""

In [2]:
import torch
import torch.nn as nn
from torch.nn import functional as F

In [3]:
class BigramLM(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.embedding_table = nn.Embedding(vocab_size, vocab_size)
        
    def forward(self, idx, targets=None):
        logits = self.embedding_table(idx)
        
        if targets is None:
            loss = None
        else:
            B,T,C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
            
        return logits, loss
    
    def generate(self, idx, max_length):
        for _ in range(max_length):
            logits, _ = self(idx)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            idx_next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, idx_next), dim=1)
        return idx

In [4]:
with open('datasets/harry_potter_books/cleaned_book.txt', 'r') as f:
    text = f.read()

In [5]:
chars = sorted(list(set(text)))
vocab_size = len(chars)

In [6]:
stoi = {ch:i for i,ch in enumerate(chars)}
itos = {i:ch for i,ch in enumerate(chars)}
encode = lambda e : [stoi[ch] for ch in e]
decode = lambda d : ''.join([itos[i] for i in d])

In [7]:
data = torch.tensor(encode(text), dtype=torch.long)

train_val_split = int(0.9*len(data))
train_data = data[:train_val_split]
val_data = data[train_val_split:]

In [8]:
torch.manual_seed(42)
block_size = 8
batch_size = 32

In [9]:
def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, size=(batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

In [10]:
model = BigramLM(vocab_size)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3)

In [11]:
for steps in range(50000):
    xb, yb = get_batch('train')
    
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())

2.4971532821655273


In [27]:
idx = torch.zeros((1,1), dtype=torch.long)
out = model.generate(idx,max_length=500)[0].tolist()
out = decode(out)
print(out)


otstak 
se. tolenis aming cly iofey yead totapavistughind 
ckllldloutat ph, tag t Hew llom tis 
" ppsas Roturwind I's oreed Har ma towe 


ate by g sstcroultht), ake 


heaind I'thit af andeeyookitharopppp hap Heay, s k!" l o " wnghuilds If h hentce g be 
Pr disloff m f te. mblemean at ferecker n t f odead 
" te 
ce welanaw tout 

Mu, Ro ho tas cofolf opton soro ghe wouedeatheatuske herarerarin trs s wantl s) brt arsco hid 
andeas FYokedid t sistha hamid 
bibrintithigheande, ok ove 
Hofim etut C
